# IMAGINE tutorial 3 -- using hammurabiX 

### Here we show how to use hammurabiX with its BregLSA field model and its CREAna CR model.

This tutorial describes the interface to hammurabiX that will be used in IMAGINE.

In [ ]:
import matplotlib
%matplotlib inline

import unittest
import os
import numpy as np
import healpy as hp

from imagine.observables.observable_dict import Measurements
from imagine.simulators.hammurabi.hammurabi import Hammurabi
from imagine.fields.breg_lsa.hamx_field import BregLSA
from imagine.fields.cre_analytic.hamx_field import CREAna
from imagine.fields.tereg_ymw16.hamx_field import TEregYMW16

The IMAGINE simulator that uses hammurabiX is the `Hammurabi` class, which in turn wraps hammurabiX's own python wrapper hampyx.  It is convenient not using hampyx directly, considering future updates in hammurabiX.  

Note that hampyx depends on an XML parameter file.  

### Generate a simulated dataset with hammurabiX

The initialization of the simulator includes defining which datasets are to be simulated.  It then prints its setup, showing that we have defined three observables:  

In [ ]:
## Create some empty measuremnt arrays
arr32 = np.zeros((1, 12*32**2))
arr16 = np.zeros((1, 12*16**2))
arr8 = np.zeros((1, 12*8**2))
measuredict = Measurements()
## Fill the measurements object with these arrays and tell
##  it what sorts of data they are.  
##
## First, a synchrotron total intensity dataset at 23 GHz at Nside=32
measuredict.append(('sync', '23', '32', 'I'), arr32)  
## Then a Faraday depth dataset of Nside=16
measuredict.append(('fd', 'nan', '16', 'nan'), arr16)
## And a Faraday dispersion measure dataset at Nside=8.
measuredict.append(('dm', 'nan', '8', 'nan'), arr8)
## It needs to k now where the base XML template is.  You may want to 
##  store different sets of hammuarbi parameters in different XML files.
xmlpath = './template.xml'
simer = Hammurabi(measurements=measuredict,xml_path=xmlpath)

Now, in addition to the types of data to simulate, the simulator also needs to be told what field models to use:


In [ ]:
## ensemble size
ensemble_size = 2

## Set up the BregLSA field with the parameters you want:
paramlist = {'b0': 6.0, 'psi0': 27.9, 'psi1': 1.3, 'chi0': 24.6}
breg_wmap = BregLSA(paramlist, ensemble_size)

## Set up the analytic CR model CREAna 
paramlist = {'alpha': 3.0, 'beta': 0.0, 'theta': 0.0,
             'r0': 5.6, 'z0': 1.2,
             'E0': 20.5,
             'j0': 0.03}
cre_ana = CREAna(paramlist, ensemble_size)


##  The free electron model based on YMW16, ie. TEregYMW16 
paramlist = dict()
fereg_ymw16 = TEregYMW16(paramlist, ensemble_size)

## Push fields to the simulator object initialized above:
simer.register_fields([breg_wmap, cre_ana, fereg_ymw16])

# Check the initialization of the simulator object:
simer._ham.print_par(['magneticfield', 'regular'])
simer._ham.print_par(['magneticfield', 'regular', 'wmap'])
simer._ham.print_par(['cre'])
simer._ham.print_par(['cre', 'analytic'])
simer._ham.print_par(['thermalelectron', 'regular'])

Now, without running the IMAGINE pipeline itself, we illustrate how hammurabiX is invoked internally by the pipeline to generate one set of observables:

In [ ]:
maps = simer([breg_wmap, cre_ana, fereg_ymw16])

`Simulator` by convention returns a `Simulations` object, which collect all required maps.  We want to get them back as arrays we can visualize with healpy.  The `data` attribute does this, and note that what it gets back is a **set** of two of each type of observable, since we specified `ensemble_size=2` above.  But since we have not yet added a random component, they are both the same:  

In [ ]:
print (maps.keys())

sync_i_raw = maps[('sync','23','32','I')].data
dm_raw = maps[('dm', 'nan', '8', 'nan')].data
fd_raw = maps[('fd', 'nan', '16', 'nan')].data

print (sync_i_raw.shape, dm_raw.shape, fd_raw.shape)

hp.mollview(sync_i_raw[0], norm='hist', cmap='jet')

### Using a random component:   BrndES

Now we add a random GMF component with the BrndES model.  This model starts with a random number generator to simulate a Gaussian random field on a cartesian grid and ensures that it is divergence free.  The grid is defined in hammurabiX XML parameter file.  

In [ ]:
from imagine.fields.brnd_es.hamx_field import BrndES

paramlist = {'rms': 6., 'k0': 0.5, 'a0': 1.7, 
             'k1': 0.5, 'a1': 0.0,
             'rho': 0.5, 'r0': 8., 'z0': 1.}
brnd_es = BrndES(paramlist, ensemble_size)

Now use the simulator to generate the maps from these field components and visualize:

In [ ]:
maps = simer([breg_wmap, brnd_es, cre_ana, fereg_ymw16])

In [ ]:
print (maps.keys())

sync_i_raw = maps[('sync','23','32','I')].data
dm_raw = maps[('dm', 'nan', '8', 'nan')].data
fd_raw = maps[('fd', 'nan', '16', 'nan')].data

print (sync_i_raw.shape, dm_raw.shape, fd_raw.shape)

In [ ]:
maps[('fd', 'nan', '16', 'nan')]._data, maps[('fd', 'nan', '16', 'nan')].data
maps[('fd', 'nan', '16', 'nan')].dtype


In [ ]:
matplotlib.rcParams['figure.figsize'] = (10.0, 4.0)
hp.mollview(sync_i_raw[0], norm='hist', cmap='jet',sub=(1,2,1),title="sim # 1")
hp.mollview(sync_i_raw[1], norm='hist', cmap='jet',sub=(1,2,2),title="sim # 2")

### This ends tutorial 3.

See more examples in the *imagine/examples* directory